# MS to EN longer text

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/ms-en-long-translation](https://github.com/huseinzol05/Malaya/tree/master/example/ms-en-long-translation).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

In [1]:
%%time
import malaya

CPU times: user 5.87 s, sys: 1.21 s, total: 7.08 s
Wall time: 9.58 s


### List available Transformer models

In [2]:
malaya.translation.ms_en.available_transformer()

,Size (MB),Quantized Size (MB),BLEU,SacreBLEU Verbose,SacreBLEU-chrF++-FLORES200,Suggested length
small,42.7,13.4,59.874731,80.6/64.3/54.1/46.3 (BP = 0.998 ratio = 0.998 ...,59.64,256
base,234,82.7,71.687583,86.2/74.8/67.2/61.0 (BP = 1.000 ratio = 1.005 ...,63.24,256
bigbird,246,63.7,59.548257,79.6/63.8/53.8/46.0 (BP = 1.000 ratio = 1.026 ...,62.49,1024
small-bigbird,50.4,13.1,55.967145,77.4/60.5/49.9/41.9 (BP = 1.000 ratio = 1.026 ...,60.57,1024
noisy-base,234,82.7,71.725493,86.3/74.8/67.2/61.0 (BP = 1.000 ratio = 1.002 ...,63.31,256


If you look at `Suggested length`, bigbird 4x longer than normal transformer models, able to infer longer text without need to partition. Let's we do some examples.

We are going to compare `small` and `base` models with `small-bigbird` and `bigbird` models for MS-EN translation task. Feel free to test on EN-MS translation.

### Load Transformer models

```python
def transformer(model: str = 'base', quantized: bool = False, **kwargs):
    """
    Load Transformer encoder-decoder model to translate MS-to-EN.

    Parameters
    ----------
    model : str, optional (default='base')
        Model architecture supported. Allowed values:

        * ``'small'`` - Transformer SMALL parameters.
        * ``'base'`` - Transformer BASE parameters.
        * ``'bigbird'`` - BigBird BASE parameters.
        * ``'small-bigbird'`` - BigBird SMALL parameters.
        * ``'noisy-base'`` - Transformer BASE parameters trained on noisy dataset.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: model
        List of model classes:

        * if `bigbird` in model, return `malaya.model.bigbird.Translation`.
        * else, return `malaya.model.tf.Translation`.
    """
```

In [3]:
transformer = malaya.translation.ms_en.transformer()
transformer_small = malaya.translation.ms_en.transformer(model = 'small')

In [4]:
bigbird = malaya.translation.ms_en.transformer(model = 'bigbird')
quantized_bigbird = malaya.translation.ms_en.transformer(model = 'bigbird', quantized = True)

In [5]:
bigbird_small = malaya.translation.ms_en.transformer(model = 'small-bigbird')
quantized_bigbird_small = malaya.translation.ms_en.transformer(model = 'small-bigbird', quantized = True)

### Long text examples

In [6]:
# https://www.bharian.com.my/berita/nasional/2021/02/785400/kabinet-tidak-pernah-bangkitkan-isu-konflik-kepentingan-najib
string = """
KUALA LUMPUR: Mahkamah Tinggi di sini, hari ini, diberitahu Kabinet tidak pernah membangkitkan isu mengenai konflik kepentingan Datuk Seri Najib Razak dalam 1Malaysia Development Berhad (1MDB).

Menurut Kod Etika Bagi Anggota Pentadbiran dan Ahli Parlimen adalah menjadi amalan ahli Kabinet untuk mengisytiharkan konflik kepentingan sekiranya mempunyai pembabitan di dalam hal yang dibincangkan dalam Mesyuarat Jemaah Menteri.

Perkara itu dimaklumkan bekas Timbalan Ketua Setiausaha (Kabinet) Bahagian Kabinet Perlembagaan dan Perhubungan Jabatan Perdana Menteri (JPM), Tan Sri Mazidah Abdul Majid, dalam keterangannya pada perbicaraan kes 1MDB yang dihadapi Najib.

Kod etika itu antara lain turut menyatakan bahawa ahli Kabinet yang mempunyai kepentingan peribadi dan bercanggah dengan kepentingan kerajaan, atau membabitkan ahli keluarga, perlu meninggalkan mesyuarat dan merekodkan ketidakhadiran mereka.

Di dalam 1MDB, Najib memegang tiga jawatan iaitu Perdana Menteri, Menteri Kewangan dan Pengerusi Pengerusi Badan Penasihat 1MDB

Menjawab soalan peguam Tan Sri Muhammad Shafee Abdullah, sama ada terdapat ahli Kabinet yang membangkitkan isu bahawa Menteri Kewangan tidak seharusnya membabitkan diri dalam perbincangan itu kerana konflik kepentingan, Maziah menjawab: "Tiada."

Muhammad Shafee: Ada sesiapa yang membangkitkan perkara berhubung hal 1MBD dengan Najib?

Maziah: Tidak

Muhammad Shafee: Timbalan Perdana Menteri (ketika itu) adalah Tan Sri Muhyiddin Yassin, manakala Datuk Seri Ahmad Husni Hanadzlah adalah bekas Menteri Kewangan II. Mereka juga tidak pernah membangkitkan hal konflik kepentingan?

Mazidah: Ya, tidak pernah.

Sementara itu, menjawab soalan Timbalan Pendakwa Raya, Ahmad Akram Gharib sama ada beliau mengetahui bahawa Najib mempunyai kepentingan peribadi dalam 1MDB, Mazidah berkata: "Tidak"

Ahmad Akram: Adakah anda mengetahui bahawa Najib secara peribadi menerima duit daripada 1MDB?

Maziah: Tidak

Ahmad Akram: Sekiranya Najib secara peribadi menerima wang daripada 1MDB adakah itu konflik kepentingan dan melanggar Kod Etika Bagi Anggota Pentadbiran dan Ahli Parlimen.

Maziah: Menurut pandangan peribadi saya, ya, namun kerana ia membabitkan menteri dan perdana menteri, saya cadangkan untuk dapatkan pandangan Peguam Negara.

Terdahulu, di awal prosiding, Maziah turut memberitahu mahkamah bahawa Najib tidak pernah menyebut nama ahli perniagaan dalam buruan, Low Taek Jho atau Jho Low pada mesyuarat Kabinet sebagai individu yang membantu beliau mendapatkan sumbangan daripada kerabat diraja Arab Saudi.

"Sekiranya perkara itu dimaklumkan kepada Kabinet, maka, ia akan dicatat dalam minit mesyuarat," katanya.

Tambah Maziah, beliau hanya mendengar dan mengetahui mengenai nama Jho Low selepas timbul isu membabitkan 1MDB.

Najib, 68, menghadapi empat pertuduhan menggunakan kedudukannya untuk memperoleh suapan berjumlah RM2.3 bilion daripada dana 1MDB dan 21 pertuduhan pengubahan wang haram membabitkan jumlah wang yang sama.

Perbicaraan di hadapan Hakim Collin Lawrence Sequerah bersambung Isnin ini.
"""

In [7]:
# https://www.astroawani.com/berita-malaysia/laporan-audit-1mdb-najib-gagal-gugurkan-sri-ram-daripada-pasukan-pendakwaan-283003
string2 = """
KUALA LUMPUR: Datuk Seri Najib Tun Razak hari ini gagal dalam satu lagi cubaannya untuk menggugurkan bekas Hakim Mahkamah Persekutuan Datuk Seri Gopal Sri Ram daripada mengetuai pasukan pendakwaan dalam kes meminda laporan audit 1Malaysia Development Berhad (1MDB) melibatkan bekas perdana menteri itu.
Ini merupakan cubaan kali ketiga Najib untuk menggugurkan Sri Ram sebagai pendakwa dalam kes jenayah berkaitan 1MDB itu. Sebelum ini, satu permohonan difailkan dalam satu lagi kes 1MDB di hadapan hakim berbeza dan cubaan kedua menerusi prosiding sivil.
Ketika menolak permohonan Ahli Parlimen Pekan itu, Hakim Mahkamah Tinggi Mohamed Zaini Mazlan berkata dakwaan Najib bahawa Sri Ram terlibat dalam siasatan terhadapnya sebagai tidak ada merit.
“Tidak ada bukti kukuh untuk menyokong dakwaan pemohon dan kekal sebagai hipotesis semata-mata. Isu ini telah disiasat oleh pihak pendakwaan selaku responden semasa permohonan pemohon (Najib) yang terdahulu.
“Isu ini sudah dibincangkan dan diputuskan. Keputusan oleh mahkamah lain sebelum ini kekal dan tidak boleh diterbalikkan,” kata hakim.
Mohamed Zaini ketika memberi alasan penolakan berkata pemohon, antara lain, menjadikan komunikasi di antara bekas Peguam Negara Tan Sri Mohamed Apandi Ali dan Sri Ram sebagai bukti berat sebelah Sri Ram terhadap pemohon dan kebimbangan pemohon berhubung perkara itu adalah tidak berasas.
“Seperti juga individu lain, Sri Ram berhak mempunyai pandangan peribadi. Itu sahaja. Namun, pertimbangan berbeza akan dibuat jika beliau menunjukkan sikap berat sebelah semasa melaksanakan tugas sebagai pendakwa raya kanan. Pandangan peribadi beliau tidak boleh dianggap akan menghalang tanggungjawab beliau sebagai pendakwa raya kanan.
“Tambahan pula, kejadian itu, seperti yang dikemukakan oleh responden, berlaku ketika sebelum pelantikan Sri Ram sebagai pendakwa raya kanan. Turut penting ialah pemohon tidak membuat sebarang aduan mengenai tindak-tanduk Sri Ram ketika menjalankan perbicaraan melibatkan pemohon. Ini mengukuhkan hujah responden bahawa Sri Ram bersikap terbuka semasa menjalankan tugas sebagai pendakwa raya kanan,” kata hakim.
Mohamed Zaini seterusnya berkata pada akhirnya, mahkamah bertanggungjawab memastikan sesebuah perbicaraan dilaksanakan secara adil demi mendapatkan keadilan.
“Mahkamah akan membantu mana-mana pihak yang dilayan secara tidak adil, jika perkara tersebut berlaku. Sehubunggan itu, permohonan pemohon ditolak,” katanya.
Perbicaraan kes laporan audit 1MDB itu akan bersambung pada 22 Feb ini.
Pada prosiding hari ini, Timbalan Pendakwa Raya Ahmad Akram Gharib bertindak bagi pihak pendakwaan, manakala Najib diwakili peguam Nur Syahirah Hanapiah.
Najib, 67, dan bekas Ketua Pegawai Eksekutif 1MDB Arul Kanda Kandasamy, 45, dibicarakan atas tuduhan meminda laporan audit 1MDB.
Ahli Parlimen Pekan itu dituduh menggunakan kedudukannya untuk mengarahkan pindaan ke atas laporan audit akhir 1MDB sebelum dibentangkan kepada Jawatankuasa Kira-Kira Wang Negara bagi mengelakkan sebarang tindakan diambil terhadapnya, sementara Arul Kanda didakwa bersubahat dengan Najib dalam membuat pindaan ke atas laporan tersebut bagi melindungi Najib daripada dikenakan tindakan.
"""

In [8]:
# https://miasa.org.my/blog/2018/08/artikel-sokongan-rakan-senasib-dapat-mengurangkan-kemasukan-semula/
string3 = """
3 Ogos 2018 – Kajian terbaharu yang dijalankan oleh University College London (UCL) mendapati khidmat penjagaan kesihatan mental daripada pekerja sokongan (support worker) yang pernah mengharungi sendiri penyakit ini mungkin boleh membantu mengurangkan kebarangkalian pesakit yang baru keluar dari penjagaan kesihatan mental akut (acute mental health care) daripada dimasukkan semula ke unit berkenaan.

 

Kertas penyelidikan tersebut yang diterbitkan pada hari ini dalam jurnal The Lancet mendapati bilangan pesakit mental yang dimasukkan semula ke unit penjagaan akut dalam tempoh setahun ialah 24% jauh lebih rendah bagi kumpulan pesakit yang ditawarkan pekerja sokongan berbentuk rakan senasib, berbanding dengan kumpulan pesakit yang hanya diberi buku kerja pemulihan peribadi sahaja.

 

“Pesakit yang dibenarkan keluar (discharge) dari perkhidmatan krisis komuniti sering dimasukkan semula ke unit penjagaan akut. Ini bukan sahaja membantutkan pemulihan, malah menggunapakai sumber yang sepatutnya dikhususkan untuk penambahbaikan jangka panjang fungsi dan kualiti hidup pesakit. Pekerja sokongan daripada golongan rakan senasib berupaya memberikan sokongan dan dorongan yang benar-benar mesra dan berempati kerana ianya datang daripada pengalaman peribadi mereka sendiri, di samping menjadi contoh tauladan (role model) yang baik untuk pemulihan pesakit,” kata pengarang utama, Profesor Sonia Johnson (Psikiatri, UCL).

Di United Kingdom (UK), lebih separuh daripada jumlah pesakit yang dimasukkan ke unit penjagaan akut dimasukkan semula ke unit berkenaan dalam tempoh setahun, namun tiada bukti kukuh yang menjelaskan kaedah untuk mengurangkan jumlah ini.

 

Sebenarnya, sokongan daripada individu yang pernah mengalami masalah kesihatan mental telah dipraktikkan dalam program seperti Pelaksanaan Pemulihan melalui Perubahan Organisasi NHS (UK) dan juga Pelan Tindakan Pemulihan Kesihatan Amerika Syarikat. Penyelidikan ini merupakan ujian terkawal rawak pertama yang seumpamanya untuk menilai keberkesanan program sokongan rakan senasib. Walaupun begitu, masih banyak penyelidikan yang perlu dijalankan sebelum strategi ini boleh dilaksanakan secara menyeluruh di UK, contohnya untuk memahami sebab-sebab mengapa program ini berkesan.

 

Pastinya, campur tangan atau intervensi dalam pengurusan kendiri (self-management) mungkin boleh membantu pesakit menguruskan kesihatan mental mereka dengan lebih baik. Dalam kajian ini, penyelidik menyediakan peserta sama ada dengan buku kerja pemulihan peribadi sahaja (220 orang) atau buku kerja berserta bantuan pekerja sokongan rakan senasib (221 orang). Peserta juga dibenarkan meneruskan penjagaan biasa mereka.

 

Kajian ini dijalankan menerusi enam buah pasukan penyelesaian krisis kesihatan mental (crisis resolution team) di England dan peserta dipilih hanya selepas mereka dibenarkan keluar dari unit krisis oleh pasukan penyelesaian krisis. Peserta kajian terdiri daripada pelbagai diagnosis antaranya termasuk skizofrenia, bipolar, psikosis, kemurungan, kecelaruan keresahan, kecelaruan stres pascatrauma (PTSD), dan kecelaruan personaliti.

 

Peserta yang menerima sokongan rakan senasib ditawarkan 10 sesi pertemuan mingguan selama satu jam. Dalam sesi pertemuan ini, pekerja sokongan mendengar masalah peserta dan bermatlamat menyemaikan harapan dengan berkongsi kemahiran dan strategi pengurusan penyakit (coping strategies) yang dikuasai sewaktu mereka sendiri berada dalam proses pemulihan. Kesemua pekerja sokongan diberikan latihan terlebih dahulu dalam kemahiran mendengar, kesedaran budaya, pendedahan kendiri, dan kerahsiaan, serta cara menggunakan buku kerja.

Rekod kesihatan peserta dipantau oleh penyelidik selama satu tahun untuk menentukan sama ada peserta dimasukkan semula ke unit penjagaan akut (seperti wad pesakit akut, pasukan penyelesaian krisis, rumah krisis, dan perkhidmatan penjagaan harian akut) atau tidak.

 

Selepas tamat tempoh satu tahun, kajian mendapati peratusan kemasukan semula ke unit penjagaan akut adalah lebih rendah bagi kumpulan peserta yang menerima intervensi berbanding dengan kelompok kawalan— dengan 29% peserta dimasukkan semula setelah menerima sokongan rakan senasib, berbanding dengan 38% peserta yang hanya menerima buku kerja.

 

Kadar penyerapan intervensi juga baik— 72% daripada peserta yang ditawarkan khidmat pekerja sokongan rakan senasib dan buku kerja menghadiri sekurang-kurangnya tiga sesi pertemuan, manakala satu pertiga menghadiri kesemua 10 sesi pertemuan.

“Kami sedia maklum bahawa banyak pengguna perkhidmatan penjagaan krisis kesihatan mental merasakan perkhidmatan ini terhenti secara mendadak kerana kurangnya penjagaan susulan yang siap sedia. Kajian kami menunjukkan bahawa pekerja sokongan rakan senasib dapat membantu mengisi kelompangan ini dengan membantu pengguna perkhidmatan (pesakit) membangunkan strategi pengurusan kendiri dan pelan pemulihan dengan cara tersendiri dan lebih bermakna, seterusnya dapat membantu pesakit terus bertahan sesudah berlakunya sesuatu krisis,” kata pengarang bersama, Dr Brynmor Lloyd-Evans (Psikiatri, UCL).
"""

do simple cleaning, remove newline and weird characters.

In [9]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string.replace('\n', ' '))).strip()

In [10]:
string = cleaning(string)
string2 = cleaning(string2)
string3 = cleaning(string3)

In [11]:
len(string.split()), len(string2.split()), len(string3.split())

(406, 433, 649)

### Translate using greedy decoder

In [12]:
from pprint import pprint

In [13]:
pprint(transformer.greedy_decoder([string]))

['KUALA LUMPUR: The High Court here today said the Cabinet had not raised the '
 "issue of Datuk Seri Najib Razak's conflict in 1Malaysia Development Berhad "
 '(1MDB). According to the Code of Ethics for Administrators and Members of '
 'Parliament it is customary practice for Cabinet members to declare conflicts '
 'of interest if they have been involved in matters discussed at the 1MDB '
 'Cabinet. The matter was announced by the former Deputy Chief Secretary '
 '(Cabinet) of the Constitutional and Liaison Division of the Prime Minister, '
 'Tan Sri Mazidah Abdul Majid, in his statement at the 1MDB hearing. The code '
 'of ethics is also stated that the Cabinet members have personal interests '
 'and conflicts with the government, or involving family members, had to leave '
 'meetings and record their absence. In 1MDB, Najib held three positions, '
 "namely the Prime Minister's Office of Finance and the Prime Minister's "
 "Chairman of the Prime Minister's Office of the Prime Minist

In [14]:
pprint(transformer_small.greedy_decoder([string]))

['KUALA LUMPUR: The High Court here today, said that the Cabinet had never '
 "raised the issue of Datuk Seri Najib Razak's interest in 1Malaysia "
 'Development Berhad (1MDB). According to the Code of Ethics For Members of '
 'the Administration and Member of Parliament is the practice of Cabinet '
 'members to declare conflict of interest if he had been discussed in matters '
 'discussed at the Cabinet. Article. The matter was informed that former '
 'Secretary-General (Cabinet) of the Cabinet, he said. Tan Sri Mazidah Abdul '
 'Majid, in his statement at the 1MDB trial, in his statement at the 1MDB '
 'case, Najib, the case of the 1MDB, he said. Najib, he had not been informed '
 'that the case of the case, Najib, he said. Najib, he said. Najib, he had not '
 'been informed that the case, he had informed that the case, Najib had '
 'informed that the 1MDB, he had not been informed that the case, he had '
 'informed that the 1MDB, he had informed that the case, he had given that he '

**Normal transformer models not able to infer long texts, we need to partition the text, example, split the text by full stop**.

In [15]:
partition_string = malaya.text.function.split_into_sentences(string)
partition_string

['KUALA LUMPUR: Mahkamah Tinggi di sini, hari ini, diberitahu Kabinet tidak pernah membangkitkan isu mengenai konflik kepentingan Datuk Seri Najib Razak dalam 1Malaysia Development Berhad (1MDB).',
 'Menurut Kod Etika Bagi Anggota Pentadbiran dan Ahli Parlimen adalah menjadi amalan ahli Kabinet untuk mengisytiharkan konflik kepentingan sekiranya mempunyai pembabitan di dalam hal yang dibincangkan dalam Mesyuarat Jemaah Menteri.',
 'Perkara itu dimaklumkan bekas Timbalan Ketua Setiausaha (Kabinet) Bahagian Kabinet Perlembagaan dan Perhubungan Jabatan Perdana Menteri (JPM), Tan Sri Mazidah Abdul Majid, dalam keterangannya pada perbicaraan kes 1MDB yang dihadapi Najib.',
 'Kod etika itu antara lain turut menyatakan bahawa ahli Kabinet yang mempunyai kepentingan peribadi dan bercanggah dengan kepentingan kerajaan, atau membabitkan ahli keluarga, perlu meninggalkan mesyuarat dan merekodkan ketidakhadiran mereka.',
 'Di dalam 1MDB, Najib memegang tiga jawatan iaitu Perdana Menteri, Menteri K

In [16]:
pprint(transformer.greedy_decoder(partition_string))

['KUALA LUMPUR: The High Court here today said the Cabinet had never raised '
 "the issue of Datuk Seri Najib Razak's conflict of interest in 1Malaysia "
 'Development Berhad (1MDB).',
 'According to the Code of Ethics for Administrative Members and Members of '
 'Parliament it is the practice of Cabinet members to declare conflicts of '
 'interest if they have any involvement in the matter discussed at the Cabinet '
 'Meeting.',
 'The matter was informed by former Deputy Secretary-General (Cabinet) of the '
 "Constitutional and Liaison Division of the Prime Minister's Department "
 '(JPM), Tan Sri Mazidah Abdul Majid, in his testimony at the 1MDB trial '
 'hearing facing Najib.',
 'The code of ethics, among others, states that Cabinet members who have '
 'personal interests and conflict with the interests of the government, or '
 'involve family members, must leave meetings and record their absence.',
 'In 1MDB, Najib held three positions namely Prime Minister, Finance Minister '
 'an

In [17]:
pprint(transformer_small.greedy_decoder(partition_string))

['KUALA LUMPUR: The High Court here today said the Cabinet had never raised '
 'the issue of conflict of interest between Datuk Seri Najib Razak in '
 '1Malaysia Development Berhad (1MDB).',
 'According to the Code of Ethics For Members of the Administration and MPs is '
 'the practice of Cabinet members to declare conflicts of interest if they '
 'have involvement in matters discussed at the Cabinet Meeting.',
 'The matter was informed by former Deputy Secretary-General (Cabinet) of the '
 "Cabinet of the Constitution and the Relations of the Prime Minister's "
 'Department (JPM), Tan Sri Mazidah Abdul Majid, in his statement at the 1MDB '
 'case trial against Najib.',
 'The code of ethics also states that Cabinet members with personal interests '
 'and conflict with the interests of the government, or involving family '
 'members, need to leave the meeting and record their absence.',
 'In 1MDB, Najib held three positions, Prime Minister, Finance Minister and '
 'Chairman of the 1MDB 

### Problem with partitioning

Problem with partition, the model assumed N + 1 element has no relationship with N element, vice versa. Attention mechanism will not work across partition.

So we introduced BigBird to solve this problem.

In [18]:
pprint(bigbird_small.greedy_decoder([string]))

['KUALA LUMPUR: The High Court here today, was told the Cabinet never raised '
 'issues about the conflict of interest of Datuk Seri Najib Razak in 1Malaysia '
 'Development Berhad (1MDB). According to the Code of Ethics for Members of '
 'Administration and Member of Parliament is the practice of Cabinet members '
 'to declare conflicts of interest if they have involvement in matters '
 'discussed in the Cabinet Meeting. The matter was informed by the former '
 'Deputy Secretary General (Cabinet) of the Cabinet of the Constitution and '
 "Relations of the Prime Minister's Department (JPM), Tan Sri Mazidah Abdul "
 'Majid, in his statement at the 1MDB case trial faced by Najib. The code of '
 'ethics, among others, also stated that Cabinet members who have personal '
 'interests and contradict the interests of the government, or involving '
 'family members, should leave the meeting and record their absence. In 1MDB, '
 'Najib holds three positions, namely the Prime Minister, Finance M

In [19]:
pprint(quantized_bigbird_small.greedy_decoder([string]))

['KUALA LUMPUR: The High Court here today, was told the Cabinet never raised '
 'issues about the conflict of interest of Datuk Seri Najib Razak in 1Malaysia '
 'Development Berhad (1MDB). According to the Code of Ethics for Members of '
 'Administration and Member of Parliament is the practice of Cabinet members '
 'to declare conflicts of interest if they have involvement in matters '
 'discussed in the Cabinet Meeting. The matter was informed by the former '
 'Deputy Secretary General (Cabinet) of the Cabinet of the Constitution and '
 "Relations of the Prime Minister's Department (JPM), Tan Sri Mazidah Abdul "
 'Majid, in his statement at the 1MDB case trial faced by Najib. The code of '
 'ethics, among others, also stated that Cabinet members who have personal '
 'interests and contradict the interests of the government, or involve family '
 'members, should leave the meeting and record their absence. In 1MDB, Najib '
 'holds three positions, namely the Prime Minister, Finance Min

In [20]:
pprint(bigbird.greedy_decoder([string]))

['KUALA LUMPUR: The High Court here today was told that the Cabinet had never '
 'raised the issue of the conflict of interest of Datuk Seri Najib Razak in '
 '1Malaysia Development Berhad (1MDB). According to the Code of Ethics for '
 'Administrative Members and Members of Parliament, it is the practice of '
 'Cabinet members to declare conflicts of interest if they have involvement in '
 'matters discussed in the Cabinet Meeting. The matter was informed by the '
 'former Deputy Secretary General (Cabinet) of the Cabinet Division of the '
 "Constitution and the Relations of the Prime Minister's Department (JPM), Tan "
 'Sri Mazidah Abdul Majid, in his testimony at the 1MDB case trial faced by '
 'Najib. The code of ethics, among others, also states that Cabinet members '
 'who have personal interests and are contrary to the interests of the '
 'government, or involving family members, have to leave the meeting and '
 'record their absence. In 1MDB, Najib held three positions, namely t

In [21]:
pprint(quantized_bigbird.greedy_decoder([string]))

['KUALA LUMPUR: The High Court here today was told that the Cabinet had never '
 'raised the issue of the conflict of interest of Datuk Seri Najib Razak in '
 '1Malaysia Development Berhad (1MDB). According to the Code of Ethics for '
 'Administrative Members and Members of Parliament, it is the practice of '
 'Cabinet members to declare conflicts of interest if they have involvement in '
 'matters discussed in the Cabinet Meeting. The matter was informed by the '
 'former Deputy Secretary General (Cabinet) of the Cabinet Division of the '
 "Constitution and the Relations of the Prime Minister's Department (JPM), Tan "
 'Sri Mazidah Abdul Majid, in his testimony at the 1MDB case trial faced by '
 'Najib. The code of ethics, among others, also states that Cabinet members '
 'who have personal interests and are contrary to the interests of the '
 'government, or involving family members, have to leave the meeting and '
 'record their absence. In 1MDB, Najib held three positions, namely t

In [22]:
pprint(bigbird_small.greedy_decoder([string2]))

['KUALA LUMPUR: Datuk Seri Najib Tun Razak today failed in another attempt to '
 'drop former Federal Court Judge Datuk Seri Gopal Sri Ram from leading the '
 'prosecution team in the case of amending the 1Malaysia Development Berhad '
 "(1MDB) audit report involving the former prime minister. This is Najib's "
 'third attempt to drop Sri Ram as prosecutor in the 1MDB-related criminal '
 'case. Earlier, an application was filed in another 1MDB case before a '
 'different judge and a second attempt through civil proceedings. While '
 'rejecting the application of the Pekan Member of Parliament, High Court '
 "Judge Mohamed Zaini Mazlan said Najib's allegation that Sri Ram was involved "
 'in the investigation into him as there was no merit. "There is no solid '
 "evidence to support the applicant's allegation and remain a mere hypothesis. "
 'This issue has been investigated by the prosecution as the previous '
 'respondent during the application of the applicant (Najib)." This issue ha

In [23]:
pprint(quantized_bigbird_small.greedy_decoder([string2]))

['KUALA LUMPUR: Datuk Seri Najib Tun Razak today failed in another attempt to '
 'drop former Federal Court Judge Datuk Seri Gopal Sri Ram from leading the '
 'prosecution team in the case of amending the 1Malaysia Development Berhad '
 "(1MDB) audit report involving the former prime minister. This is Najib's "
 'third attempt to drop Sri Ram as prosecutor in the 1MDB-related criminal '
 'case. Earlier, an application was filed in another 1MDB case before a '
 'different judge and a second attempt through civil proceedings. While '
 'rejecting the application of the Pekan Member of Parliament, High Court '
 "Judge Mohamed Zaini Mazlan said Najib's allegation that Sri Ram was involved "
 'in the investigation into him as there is no merit. "There is no solid '
 "evidence to support the applicant's allegation and remain a mere hypothesis. "
 'This issue has been investigated by the prosecution as the previous '
 'respondent during the application of the applicant (Najib)." This issue has

In [24]:
pprint(bigbird.greedy_decoder([string2]))

['KUALA LUMPUR: Datuk Seri Najib Tun Razak today failed in another attempt to '
 'drop former Federal Court Judge Datuk Seri Gopal Sri Ram from leading the '
 'prosecution team in the case of amending the 1Malaysia Development Berhad '
 "(1MDB) audit report involving the former prime minister. This is Najib's "
 'third attempt to drop Sri Ram as a prosecution in the 1MDB-related criminal '
 'case. Previously, an application was filed in another 1MDB case before a '
 'different judge and a second attempt through civil proceedings. While '
 "rejecting the Pekan Member of Parliament's application, High Court Judge "
 "Mohamed Zaini Mazlan said Najib's allegation that Sri Ram was involved in "
 'his investigation into him as no merit. "There is no solid evidence to '
 "support the applicant's allegations and remain a mere hypothesis. This issue "
 'has been investigated by the prosecution as the respondent during the '
 'application of the applicant (Najib) earlier." This issue has been di

In [25]:
pprint(quantized_bigbird.greedy_decoder([string2]))

['KUALA LUMPUR: Datuk Seri Najib Tun Razak today failed in another attempt to '
 'drop former Federal Court Judge Datuk Seri Gopal Sri Ram from leading the '
 'prosecution team in the case of amending the 1Malaysia Development Berhad '
 "(1MDB) audit report involving the former prime minister. This is Najib's "
 'third attempt to drop Sri Ram as a prosecution in the 1MDB-related criminal '
 'case. Previously, an application was filed in another 1MDB case before a '
 'different judge and a second attempt through civil proceedings. While '
 "rejecting the Pekan Member of Parliament's application, High Court Judge "
 "Mohamed Zaini Mazlan said Najib's allegation that Sri Ram was involved in an "
 'investigation into him as no merit. "There is no solid evidence to support '
 "the applicant's allegations and remain a mere hypothesis. This issue has "
 'been investigated by the prosecution as the respondent during the applicant '
 '(Najib) application." This issue has been discussed and deci

In [21]:
pprint(bigbird_small.greedy_decoder([string3]))

['3 August 2018 - The latest study conducted by University College London '
 '(UCL) found that mental healthcare services from support workers who have '
 'been facing the disease themselves may help reduce the probability of '
 'patients who have just left out of acute mental health care from '
 're-introduced to the unit. The research paper published today in the journal '
 'The Lancet found that the number of mental patients re-induced into acute '
 'care units within a year was 24% much lower for the group of patients '
 'offered by support workers in the form of senasib friends, compared to '
 'groups of patients who are only given personal rehabilitation work books. '
 '"The pain allowed to go out (discharge) from community crisis services is '
 'often re-entered into acute care units. This not only helps recovery, but '
 'also uses resources that should be reserved for long-term improvement of '
 "patients' functions and quality of life. Support workers from the group of "
 'sen

In [22]:
pprint(quantized_bigbird_small.greedy_decoder([string3]))

['3 August 2018 - The latest study conducted by University College London '
 '(UCL) found that mental healthcare services from support workers who have '
 'been going through the disease themselves may help reduce the probability of '
 'patients who have just left from acute mental health care from re-increased '
 'to the unit. The research paper published today in the journal The Lancet '
 'found that the number of mental patients re-induced into acute care units '
 'within a year is 24% much lower for the group of patients offered by support '
 'workers in the form of dustry partners, compared to groups of patients who '
 'are only given personal rehabilitation work books. "The pain allowed to go '
 'out (discharge) from community crisis services is often re-entered to acute '
 'care units. This not only hinders recovery, but also uses resources that '
 "should be reserved for long-term improvement of patients' functions and "
 'quality of life. Support workers from the group of sena

In [26]:
pprint(bigbird.greedy_decoder([string3]))

['3 August 2018 - A recent study conducted by University College London (UCL) '
 'found that mental healthcare services from support workers who have '
 'undergone the disease themselves may help reduce the probability of patients '
 'who have just left acute mental health care (acute mental health care) from '
 'being re-entered to the unit. The research paper published today in the '
 'journal The Lancet found that the number of mental patients re-entered to '
 'acute care units within a year was 24% much lower for the group of patients '
 'offered by support workers in the form of senib friends, compared to the '
 'group of patients who were only given personal rehabilitation workbooks '
 'only. "Patients allowed to leave (discharge) from community crisis services '
 'are often re-entered to acute care units. This not only hinders recovery, '
 'but also uses resources that should be reserved for long-term improvement of '
 'patient function and quality of life. Support workers from 

In [27]:
pprint(quantized_bigbird.greedy_decoder([string3]))

['August 3, 2018 - A recent study conducted by University College London (UCL) '
 'found that mental healthcare services from support workers who have '
 'undergone the disease themselves may help reduce the probability of patients '
 'who have just left acute mental health care (acute mental health care) from '
 'being re-entered to the unit. The research paper published today in the '
 'journal The Lancet found that the number of mental patients re-entered into '
 'acute care units within a year was 24% much lower for the group of patients '
 'offered by support workers in the form of senib partners, compared to the '
 'group of patients who were only given personal rehabilitation workbooks '
 'only. "Puncharge allowed out of community crisis services are often '
 're-entered to acute care units. This not only hinders recovery, but also '
 'uses resources that should be reserved for long-term improvement of patient '
 'function and quality of life. Support workers from the senasib pa